# GAN example with PyTorch [NOT WORKING]

This is intended to be a concise example of Generative Adversarial Networks, therefore no deep explanation is given.

In GAN architecture, there are two Neural Networks competing against each other. The first is called generator (we'll give it the $G$ label) and it simply generate new data. The second is a discriminative network (we'll label it as $D$) that is a common classificator.

The generator $G$ goal is to fool the discriminator $D$, trying to make it think the generated data are real. The discriminator will try to classify incoming data as real or fake. This is how the $G$ and $D$ compete. 

Code taken from [this article](https://medium.com/@devnag/generative-adversarial-networks-gans-in-50-lines-of-code-pytorch-e81b79659e3f)

In [54]:
# importing libraries
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

**1.** First we create the function to generate our dataset.

In [55]:
def get_distribution_sampler(mu, sigma):
    return lambda n: torch.Tensor(np.random.normal(mu, sigma, (1, n)))

**2.** Next we prepare function that creates the input that goes to the generator.

In [56]:
def get_generator_input_sampler():
    return lambda m, n: torch.rand(m, n)

**3.** Next, we create our generator (which is a standard FeedFoward network).

Here, `nn.Linear` is a linear layer from PyTorch that applies a linear transformation in the data: $y = xA^T + b$.

In [57]:
class Generator(nn.Module):
    
    """
    Defines the generator network, G.
    """
    
    def __init__(self, input_size, hidden_size, output_size, f):
        super(Generator, self).__init__()
        self.map1 = nn.Linear(input_size, hidden_size)
        self.map2 = nn.Linear(hidden_size, hidden_size)
        self.map3 = nn.Linear(hidden_size, output_size)
        self.f = f
        
    def foward(self, x):
        x = self.f(self.map1(x))
        x = self.f(self.map2(x))
        return self.f(self.map3(x))

**4.** Now we create our discriminator (also, a FeedFoward network). Note that the code is the same.

In [58]:
class Discriminator(nn.Module):
    
    """
    Defines the discriminator network, D.
    """
    
    def __init__(self, input_size, hidden_size, output_size, f):
        super(Discriminator, self).__init__()
        self.map1 = nn.Linear(input_size, hidden_size)
        self.map2 = nn.Linear(hidden_size, hidden_size)
        self.map3 = nn.Linear(hidden_size, output_size)
        self.f = f
        
    def forward(self, x):
        x = self.f(self.map1(x))
        x = self.f(self.map2(x))
        return self.f(self.map3(x))

**5.** Finally, we create the training loop.

To start the loop, we need to set some things up:

In [59]:
def get_moments(d):
    """
    Return the first 4 moments of the data provided
    For more information about moments check the link:
    https://www.thoughtco.com/what-are-moments-in-statistics-3126234
    """
    
    mean = torch.mean(d)
    diffs = d - mean
    var = torch.mean(torch.pow(diffs, 2.0))
    std = torch.pow(var, 0.5)
    zscores = diffs / std
    skews = torch.mean(torch.pow(zscores, 3.0))
    kurtoses = torch.mean(torch.pow(zscores, 4.0)) - 3.0  # excess kurtosis, should be 0 for Gaussian
    final = torch.cat((mean.reshape(1,), std.reshape(1,), skews.reshape(1,), kurtoses.reshape(1,)))
    return final

# Data params
data_mean = 4
data_stdev = 1.25

g_input_size = 1                # Random noise dimension coming into generator, per output vector
g_hidden_size = 5               # Generator hidden layers size
g_output_size = 1               # Size of generated output vector

d_input_size = 4                # Input size coming into discriminator
d_hidden_size = 10              # Discriminator hidden layers size
d_output_size = 1               # Single dimension for 'real' vs. 'fake' classification

epochs = 5000                   # Training epochs
d_steps = 20                    # Train steps for D
g_steps = 20                    # Train steps for G

minibatch_size = d_input_size   #

# Get samplers
g_sampler = get_generator_input_sampler()
d_sampler = get_distribution_sampler(data_mean, data_stdev)

# Instantiates generator
G = Generator(input_size=g_input_size,
                  hidden_size=g_hidden_size,
                  output_size=g_output_size,
                  f=torch.tanh)

# Instantiates discriminator
D = Discriminator(input_size=d_input_size,
                  hidden_size=d_hidden_size,
                  output_size=d_output_size,
                  f=torch.sigmoid)

d_learning_rate = 1e-3          # D learning rate
g_learning_rate = 1e-3          # G learning rate
sgd_momentum = 0.9              #
    
# Define optimizers for D and G
d_optimizer = optim.SGD(D.parameters(), lr=d_learning_rate, momentum=sgd_momentum)
g_optimizer = optim.SGD(G.parameters(), lr=g_learning_rate, momentum=sgd_momentum)

# Define the cost function as the
# binary cross-entropy function.
cost = nn.BCELoss()

dfe, dre, ge = 0, 0, 0
d_real_data, d_fake_data, g_fake_data = None, None, None

Then, we perform the actually training:

In [60]:
for epoch in range(epochs):
    
    # Discriminator loop
    for d_index in range(d_steps):
        D.zero_grad()
        
        # Train D on real data
        d_real_data = Variable(d_sampler(d_input_size))
        d_real_decision = D(get_moments(d_real_data))
        d_real_error = cost(d_real_decision, Variable(torch.ones([1, 1])))
        d_real_error.backward()
        
        # Train D on fake data
        d_gen_input = Variable(g_sampler(minibatch_size, g_input_size))
        d_fake_data = G(d_gen_input).detach()  # detach to avoid G training with these data
        d_fake_decision = D(get_moments(d_fake_data.t()))
        d_fake_error = cost(d_fake_decision, Variable(torch.ones([1, 1])))
        d_fake_error.backward()
        d_optimizer.step() # chages D's parameters based on backpropagation results
        dre = d_real_error.data.storage().tolist()[0]
        dfe = d_fake_error.data.storage().tolist()[0]
        
    # Generator loop
    for g_index in range(g_steps):
        G.zero_grad()
        
        gen_input = Variable(g_sampler(minibatch_size, g_input_size))
        g_fake_data = G(gen_input)
        dg_fake_decision = D(get_moments(g_fake_data.t()))
        g_error = cost(dg_fake_decision, Variable(torch.ones([1, 1])))
        
        g.backward()
        g_optimizer.step()
        ge = g_error.data.storage().tolist()[0]
        

/home/jean/.local/lib/python3.6/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


NotImplementedError: 